In [1]:
from urllib.parse import urlparse, parse_qsl, parse_qs
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import gzip
import codecs
from tqdm import tqdm
import sys

from multiprocessing.dummy import Pool, Queue

In [4]:
list_refs = []
s = 'https://gg.deals/games/?sort=metascore&type=1&page='
number_hrefs = 0
i = 0
with tqdm(total=300) as pbar:
    while number_hrefs < 300:
        s_tmp = s + str(i)
        i += 1

        r_get = get_page(s_tmp)
        if not r_get:
            continue
        soup = BeautifulSoup(r_get.content, "lxml")
        # print(type(r_get.text))
        # print(re.findall( r'with-badges game-id-\d+ game-list-item item item-list grid-layout with-full-link with-game-collections-buttons', r_get.text))
        for line in re.findall( r'with-badges game-id-\d+ game-list-item item item-list grid-layout with-full-link with-game-collections-buttons', r_get.text):
            g_data = soup.find_all("div", {"class": line})
            if not g_data:
                continue
            list_refs.append('http://gg.deals' + g_data[0].a.get('href'))
            number_hrefs += 1
            pbar.update(1)
            if number_hrefs == 300:
                break
            # print (g_data[0].a.get('href'))
        # print(number_hrefs)
        g_data = soup.find_all("div", {"class": ['with-badges game-id']})


100%|██████████| 300/300 [00:25<00:00, 11.58it/s]


In [ ]:
s = "with-badges game-id-98969"
print(re.findall( r'with-badges game-id-\d', s))
list_refs[0] = ""
print(list_refs[0])

In [40]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    for i in range(n_attempts):
        try:
            r_get = requests.get(url, **kwargs)
        except Exception:
            continue
        if r_get.status_code == 200:
            return r_get
        time.sleep(t_sleep)
    return None


In [6]:
r_get = get_page('https://gg.deals/game/half-life-2/')
soup = BeautifulSoup(r_get.content, "lxml")
print (r_get.status_code)

200


In [7]:
def find_name(soup):
    g_data = soup.find_all("div", {"class": "breadcrumbs-widget pjax-inner-replace"})
    if not g_data:
        return None
    # print(g_data[0].find_all("li")[-1].span.text)
    return g_data[0].find_all("li")[-1].span.text


In [8]:
def find_image(soup):
    g_data = soup.find_all("div", {"class": "game-info-image"})
    if not g_data:
        return None
    # print(g_data[0].img.get("src"))
    return g_data[0].img.get("src")
    
find_image(soup)

'https://img.gg.deals/ac/4d/26d9e40fcbbd8415599725556f465016fa61_307xt176.jpg'

In [9]:
def find_href(soup):
    g_data = soup.find_all("a", {"class": "game-link-widget"})
    if not g_data:
        return None
    # print(g_data[0].get("href"))
    return g_data[0].get("href")
    
find_href(soup)

'https://gg.deals/redirect/9107e47eabe255a536cbb7bbef321f5004a4460c/?utm_source=games%2Fsingle'

In [10]:
def find_wish(soup):
    g_data = soup.find_all("div", {"class": re.findall("r'wishlisted-game wishlisted-game-\d+ game-action-wrap", str(soup))})
    # print(g_data)
    if not g_data:
        return None
    data_tmp = g_data[0].find_all("span", {"class":'count'})
    if not data_tmp:
        return None
    # print(data_tmp[0].text)
    return data_tmp[0].text

find_wish(soup)

'515'

In [11]:
def find_alert(soup):
    g_data = soup.find_all("div", {"class": re.findall("alerted-game alerted-game-\d+ game-action-wrap", str(soup))})
    if not g_data:
        return None
    # print(g_data[0].find_all("span", {"class":'count'})[0].text)
    data_tmp = g_data[0].find_all("span", {"class":'count'})
    if not data_tmp:
        return None
    return data_tmp[0].text

find_alert(soup)

'84'

In [12]:
def find_own(soup):
    g_data = soup.find_all("div", {"class": re.findall("owned-game owned-game-\d+ game-action-wrap", str(soup))})
    if not g_data:
        return None
    data_tmp = g_data[0].find_all("span", {"class":'count'})
    # print(data_tmp[0].text)
    return data_tmp[0].text

find_own(soup)

'17199'

In [13]:
def find_packs(soup):
    list_packs=[]
    g_data = soup.find_all("div", {"class": "game-card ab-alternative-2"})
    if not g_data:
        return []
    data_tmp = g_data[0].find_all("div", {"class": "game-section game-offers"})[0]
    if not data_tmp:
        return []
    # print(g_data[0].div)\
    data_tmp = data_tmp.find_all("div", {"class": "games-box games-box-related-list load-more-content"})
    if len(data_tmp) < 2:
        return []
    for str in data_tmp[1].find_all("a", {"class":"full-link"}):
        list_packs.append("https://gg.deals" + str.get("href"))
    # g_list = g_data[0].find_all("div", {"class": "game-list-item game-item game-full-hoverable with-full-link game-box-options with-game-collections-buttons related-layout game-id-43031"})
    return list_packs

find_packs(soup)

['https://gg.deals/ru/pack/valve-complete-pack/',
 'https://gg.deals/ru/pack/the-orange-box/',
 'https://gg.deals/ru/pack/half-life-complete/',
 'https://gg.deals/ru/pack/half-life-2---holiday-2006-collection/']

In [14]:
def find_dlcs(soup):
    list_dlcs = []
    g_data = soup.find_all("div", {"class": "game-card ab-alternative-2"})
    # print(g_data)
    if not g_data:
        return []
    data_tmp = g_data[0].find_all("div", {"class": "game-related-container list-more-wrapper list-more-ajax-wrapper load-more-container"})
    if not data_tmp:
        return []
    for line in data_tmp[0].find_all("a", {"class": "full-link"}):
        list_dlcs.append("gg.deals" + line.get("href"))
    return list_dlcs

find_dlcs(soup)

['gg.deals/ru/dlc/half-life-2-soundtrack/']

In [15]:
def find_pc(soup):
    list_pc = []
    g_data = soup.find_all("div", {"class":"game-requirements-tabs"})
    if not g_data:
        return []
    g_data = g_data[0].find_all("li")
    if not g_data:
        return []
    for line in g_data:
        list_pc.append(str(line.text))
    return list_pc

find_pc(soup)

['Windows', 'Mac']

In [16]:
def get_data(soup):
    g_data = soup.find_all("div", {"class":"game-info-details-section game-info-details-section-release"})
    if not g_data:
        return None
    # print (g_data[0].p.text)
    return g_data[0].p.text
    
get_data(soup)

'16 Nov 2004'

In [17]:
def get_developer(soup):
    g_data = soup.find_all("div", {"class":"game-info-details-section game-info-details-section-developer"})
    if not g_data:
        return None
    # print (g_data[0].p.text)
    return g_data[0].p.text
    
get_developer(soup)

'Valve'

In [18]:
def get_metascore(soup):
    g_data = soup.find_all("div", {"class":"game-info-details-content"})
    if not g_data:
        return None
    data_tmp = g_data[0].find_all("div", {"class":"score-col"})
    if not data_tmp:
        return None
    ans = data_tmp[0].span.text
    # print (ans)
    return ans


r_get = requests.get('https://gg.deals/game/mission-its-complicated/')
soup = BeautifulSoup(r_get.content, "lxml")
get_metascore(soup)

'91'

In [19]:
def get_userscore(soup):
    g_data = soup.find_all("div", {"class":"game-info-details-content"})
    # print(g_data)
    if not g_data:
        return None
    data_tmp = g_data[0].select('div[class="score-col"]')
    if (len(data_tmp) < 2):
        return None
    ans = data_tmp[1].span.text
    # print (ans)
    return ans



r_get = requests.get('https://gg.deals/game/mission-its-complicated/')
soup = BeautifulSoup(r_get.content, "lxml")
get_userscore(soup)

In [20]:
def get_genres(soup):
    list_genres = []
    g_data = soup.find_all("div", {"class":"tags-list link-list"})
    if not g_data:
        return []
    for line in g_data[0].find_all("a"):
        list_genres.append(line.text)
    # print(list_genres)
    return list_genres
    
get_genres(soup)

['Adventure', 'Casual', 'Indie']

In [21]:
def get_tags(soup):
    list_tags = []
    g_data = soup.find_all("div", {"class":"tags-list tags-list-dotdotdot"})
    for line in g_data[0].find_all("a"):
        list_tags.append(line.text)
    # print(list_tags)
    return list_tags
    
get_tags(soup)

['Casual',
 'Superhero',
 'Visual Novel',
 'Romance',
 'Dating Sim',
 'Choices Matter',
 'LGBTQ+',
 'Story Rich',
 'Adventure',
 'Indie',
 'Funny',
 'Free to Play',
 'Singleplayer',
 'Action',
 'Arcade',
 'Exploration',
 'Turn-Based Tactics',
 '2D',
 'Anime',
 'Cartoon']

In [22]:
def get_features(soup):
    list_features = []
    g_data = soup.find_all("div", {"class":"tags-list tags-list-dotdotdot"})
    if (len(g_data) < 2):
        return []
    for line in g_data[1].find_all("a"):
        list_features.append(line.text)
    # print(list_features)
    return list_features

get_features(soup)

['Single-player', 'Steam Achievements', 'Steam Cloud']

In [23]:
def get_label_count(soup):
    g_data = soup.find_all("div", {"class":"score-heading"})
    if not g_data:
        return None
    data_tmp = g_data[0].find_all("a")
    if not data_tmp:
        return None
    data_tmp = data_tmp[0].find_all("span", {"class":"reviews-label"})
    if not data_tmp:
        return None
    ans = data_tmp[0].text
    ans = ans.replace(' (', '_ ')
    ans = ans.replace(')', '_ ')
    ans = ans.replace(',', '')
    return re.split('_ ', ans)[0:-1]
    
get_label_count(soup)

['Very Positive', '122']

In [24]:
def get_pctg(soup):
    # g_data = soup.find_all("div", {"class":"score-heading"})
    g_data = soup.find_all("span", {"class" : re.findall("score-bar score-steam", str(soup))})
    if not g_data:
        return None
    data_tmp = g_data[0].find_all("span")
    if not data_tmp:
        return None
    data_tmp = data_tmp[0].get("class")
    if (len(data_tmp) < 2):
        return None
    ans = re.findall(r'\d+', data_tmp[1])
    # print(int(re.findall(r'\d+', ans)[0]))
    if not ans:
        return None
    return int(ans[0])
    
get_pctg(soup)

96

In [25]:
r_get = requests.get('https://gg.deals/ru/games/chartHistoricalData/1549/?hideKeyshops=0')
# print((r_get.content))
# response = json.loads(requests.get("https://gg.deals/ru/games/chartHistoricalData/1549/?hideKeyshops=1").text)

In [26]:
def get_price_history(soup):
    list_prices = []
    g_data = soup.find_all("div", {"id": 'historical-chart-container'})
    if not g_data:
        return []
    url = g_data[0].get("data-with-keyshops-url")
    r_get = get_page('https://gg.deals' + url, headers={'x-requested-with': 'XMLHttpRequest'})
    if not r_get:
        return []
    json_data = json.loads(r_get.text)
    # print(json_data['chartData']['deals'])
    if not json_data['chartData'] or not json_data['chartData']['deals']:
        return []
    for list_iter in json_data['chartData']['deals']:
        list_prices.append({'ts': int(list_iter['x'] / 1000), 'price': list_iter['y'], 'shop': list_iter['shop']})
    return list_prices

get_price_history(soup)

[{'ts': 1601506940, 'price': 0, 'shop': 'Steam'},
 {'ts': 1619946497, 'price': 0, 'shop': 'Steam'}]

In [ ]:
def process_page(href):
    r_get = get_page(href)
    if  not r_get:
        # print(href + "problems with")
        return None
    soup = BeautifulSoup(r_get.content, "lxml")
    result = {
        "url": href,
        "name": find_name(soup),
        "image": find_image(soup),
        "release_date": get_data(soup),
        "developer": get_developer(soup),
        "metacritic_score": int(get_metascore(soup)) if get_metascore(soup) else None,
        "user_score": float(get_userscore(soup)) if get_userscore(soup) else None,
        "review_label": get_label_count(soup)[0] if get_label_count(soup) else None,
        "review_positive_pctg": get_pctg(soup),
        "review_count": int(get_label_count(soup)[1]) if get_label_count(soup) else None,
        "genres": get_genres(soup),
        "tags": get_tags(soup),
        "features": get_features(soup),
        "pc_systems": find_pc(soup),
        "wishlist_count": int(find_wish(soup)),
        "alert_count": int(find_alert(soup)),
        "owners_count": int(find_own(soup)),
        "dlcs": find_dlcs(soup),
        "packs": find_packs(soup),
        "market_url": find_href(soup),
        "price_history": get_price_history(soup)
    }
    return {k: v for k, v in result.items() if v is not None}

# print(list_refs[219])
# result = process_page('https://gg.deals/game/hades/')
# json.dumps(result, ensure_ascii=False)
# print(json.dumps(result, indent=4))

In [28]:
len(list_refs)

300

In [37]:
for href_iter in list_refs[218:219]:
    # print (list_refs[42])
    try:
        json_tmp = process_page(href_iter)
    except Exception:
        print("Problems with\t", end='')
        print(href_iter)
        raise Exception
    with open('result.txt', 'a') as f_output:
        f_output.write(json.dumps(json_tmp))
        f_output.write('\n')
        f_output.close()

In [3]:


queue = Queue(list_refs)   # очередь ссылок на книги


In [55]:
import gzip
import codecs
from tqdm import tqdm
from multiprocessing.dummy import Pool, Queue

queue = Queue()   # очередь ссылок на книги
for list_tmp in list_refs:
    queue.put(list_tmp)
def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        with open('{:01d}.txt'.format(i), mode='w') as f_output:
            while not queue.empty():
                href_tmp = queue.get()
                # print(href_tmp + " ", end="")
                record = process_page(href_tmp)
                if not record:
                    message = "Problems with " + href_tmp + " . Can't download this page."
                    with lock:
                        print(message, file=sys.stderr)
                        # print(message)
                        pbar.update(1)
                    continue
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
                # счетчик должен атомарно обновиться
                with lock:
                    pbar.update(1)
            f_output.close()
            

with Pool(processes=4) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

100%|██████████| 300/300 [06:22<00:00,  1.28s/it]
